
# Previsão de preços de casas:um exemplo de regressão

Este bloco de notas contém os exemplos de código encontrados no Capítulo 3, Seção 6 do Deep Learning with Python. Observe que o texto original apresenta muito mais conteúdo, em particular mais explicações e figuras: neste bloco de notas, você encontrará apenas o código-fonte e comentários relacionados.

Em nossos dois exemplos anteriores, estávamos considerando problemas de classificação, em que o objetivo era prever um único rótulo discreto de um ponto de dados de entrada. Outro tipo comum de problema de aprendizado de máquina é a "regressão", que consiste em prever um valor contínuo em vez de um rótulo discreto. Por exemplo, prever a temperatura amanhã, dados dados meteorológicos, ou prever o tempo que um projeto de software levará para ser concluído, dadas suas especificações.

Não misture "regressão" com o algoritmo "regressão logística": confusamente, "regressão logística" não é um algoritmo de regressão, é um algoritmo de classificação.


**O conjunto de dados do Boston Housing Price**
Estaremos tentando prever o preço médio das casas em um determinado subúrbio de Boston em meados da década de 1970, dados alguns pontos de dados sobre o subúrbio na época, como a taxa de criminalidade, a taxa de imposto sobre a propriedade local, etc.

O conjunto de dados que usaremos tem outra diferença interessante de nossos dois exemplos anteriores: ele tem muito poucos pontos de dados, apenas 506 no total, dividido entre 404 amostras de treinamento e 102 amostras de teste, e cada "recurso" nos dados de entrada (por exemplo, taxa de criminalidade é uma característica) tem uma escala diferente. Por exemplo, alguns valores são proporções, que assumem valores entre 0 e 1, outros assumem valores entre 1 e 12, outros entre 0 e 100 ...

Vamos dar uma olhada nos dados:

In [1]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

57344/57026 [==============================] - 0s 0us/step


In [2]:
train_data.shape

(404, 13)

In [3]:
test_data.shape

(102, 13)

Como você pode ver, temos 404 amostras de treinamento e 102 amostras de teste. Os dados são compostos por 13 recursos. Os 13 recursos nos dados de entrada são os seguintes:

*   Taxa de criminalidade per capita.
*   Proporção de terrenos residenciais zoneados para lotes com mais de 25.000 pés quadrados.
*   Proporção de acres de negócios não varejistas por cidade.
*   Variável dummy de Charles River (= 1 se a área limita o rio; 0 caso contrário).
*   Número médio de cômodos por moradia.
*   Proporção de unidades ocupadas pelo proprietário construídas antes de 1940.
*  Distâncias ponderadas para cinco centros de empregos de Boston.
*  Índice de acessibilidade às rodovias radiais.
*  Taxa de imposto de propriedade de valor total por $ 10.000.
*  Proporção aluno-professor por cidade.
*  1000 * (Bk - 0,63) ** 2 onde Bk é a proporção de negros por cidade.
*  % de status inferior da população.

As metas são os valores medianos das casas ocupadas pelo proprietário, em milhares de dólares:


In [4]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13


# Preparando os dados

Seria problemático alimentar os valores de uma rede neural em que todos assumissem intervalos totalmente diferentes. A rede pode ser capaz de se adaptar automaticamente a esses dados heterogêneos, mas certamente tornaria o aprendizado mais difícil. Uma prática recomendada amplamente difundida para lidar com esses dados é fazer a normalização por recurso: para cada recurso nos dados de entrada (uma coluna na matriz de dados de entrada), vamos subtrair a média do recurso e dividir pelo desvio padrão, que o recurso será centralizado em torno de 0 e terá um desvio padrão da unidade. Isso é feito facilmente no Numpy:


In [5]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

Observe que as quantidades que usamos para normalizar os dados de teste foram calculadas usando os dados de treinamento. Nunca devemos usar em nosso fluxo de trabalho qualquer quantidade calculada nos dados de teste, mesmo para algo tão simples como a normalização de dados.


**Construindo nossa rede**
Como há poucas amostras disponíveis, usaremos uma rede muito pequena com duas camadas ocultas, cada uma com 64 unidades. Em geral, quanto menos dados de treinamento você tiver, pior será o overfitting, e usar uma rede pequena é uma forma de mitigar o overfitting.

In [6]:
from keras import models
from keras import layers

def build_model():
    # Porque precisaremos instanciar
    # o mesmo modelo várias vezes,
    # usamos uma função para construí-lo.
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

Nossa rede termina com uma única unidade, e nenhuma ativação (ou seja, será uma camada linear). Esta é uma configuração típica para regressão escalar (ou seja, regressão em que tentamos prever um único valor contínuo). A aplicação de uma função de ativação restringiria o intervalo que a saída pode assumir; por exemplo, se aplicássemos uma função de ativação sigmóide à nossa última camada, a rede só poderia aprender a prever valores entre 0 e 1. Aqui, como a última camada é puramente linear, a rede é livre para aprender a prever valores em qualquer faixa.

Observe que estamos compilando a rede com a função de perda mse - Erro Quadrático Médio, o quadrado da diferença entre as previsões e as metas, uma função de perda amplamente usada para problemas de regressão.

Também estamos monitorando uma nova métrica durante o treinamento: mae. Isso significa erro médio absoluto. É simplesmente o valor absoluto da diferença entre as previsões e os alvos. Por exemplo, um MAE de 0,5 neste problema significaria que nossas previsões estão erradas em \ $ 500 em média.

Validando nossa abordagem usando validação K-fold
Para avaliar nossa rede enquanto ajustamos seus parâmetros (como o número de épocas usadas para o treinamento), poderíamos simplesmente dividir os dados em um conjunto de treinamento e um conjunto de validação, como estávamos fazendo em nossos exemplos anteriores. No entanto, como temos tão poucos pontos de dados, o conjunto de validação acabaria sendo muito pequeno (por exemplo, cerca de 100 exemplos). Uma consequência é que nossas pontuações de validação podem mudar muito dependendo de quais pontos de dados escolhemos usar para validação e quais escolhemos para treinamento, ou seja, as pontuações de validação podem ter uma alta variação em relação à divisão de validação. Isso nos impediria de avaliar de forma confiável nosso modelo.

A melhor prática em tais situações é usar a validação cruzada K-fold. Consiste em dividir os dados disponíveis em K partições (normalmente K = 4 ou 5), instanciar K modelos idênticos e treinar cada um nas partições K-1 enquanto avalia a partição restante. A pontuação de validação para o modelo usado seria então a média das pontuações de validação K obtidas.

Em termos de código, isso é simples:

In [7]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare os dados de validação: dados da partição # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare os dados de treinamento: dados de todas as outras partições
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Construir o modelo Keras (já compilado)
    model = build_model()
    # Treine o modelo (no modo silencioso, verboso = 0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    # Avalie o modelo nos dados de validação
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [8]:
all_scores

[1.9392619132995605, 2.5057201385498047, 2.751328229904175, 2.487156391143799]

In [9]:
np.mean(all_scores)

2.4208666682243347

Como você pode notar, as diferentes execuções realmente mostram pontuações de validação bastante diferentes, de 2.1 a 2.9. Sua média (2,4) é uma métrica muito mais confiável do que qualquer uma dessas pontuações - esse é o ponto principal da validação cruzada K-fold. Neste caso, perdemos em média \ $ 2.400, o que ainda é significativo considerando que os preços variam de \ $ 10.000 a \ $ 50.000.


Vamos tentar treinar a rede um pouco mais: 500 épocas. Para manter um registro do desempenho do modelo em cada época, modificaremos nosso loop de treinamento para salvar o log de pontuação de validação por época:


In [10]:
from keras import backend as K

# Alguma limpeza de memória
K.clear_session()

In [12]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    # Prepare os dados de validação: dados da partição # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare os dados de treinamento: dados de todas as outras partições
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Construir o modelo Keras (já compilado)
    model = build_model()
    # Treine o modelo (no modo silencioso, verboso = 0)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3



Podemos então calcular a média das pontuações MAE por época para todas as dobras:

In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:

import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()


Pode ser um pouco difícil ver o gráfico devido a problemas de escala e variação relativamente alta. Vamos:

Omita os primeiros 10 pontos de dados, que estão em uma escala diferente do resto da curva.
Substitua cada ponto por uma média móvel exponencial dos pontos anteriores, para obter uma curva suave.

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()


De acordo com este gráfico, parece que a validação MAE para de melhorar significativamente após 80 épocas. Passado esse ponto, começamos a overfitting.

Assim que terminarmos de ajustar outros parâmetros do nosso modelo (além do número de épocas, também podemos ajustar o tamanho das camadas ocultas), podemos treinar um modelo de "produção" final em todos os dados de treinamento, com os melhores parâmetros, em seguida, observe seu desempenho nos dados de teste:

In [ ]:
# Obtenha um novo modelo compilado.
model = build_model()
# Treine-o com todos os dados.
model.fit(train_data, train_targets,
          epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mae_score